In [142]:
import os
import glob
import re
import pandas as pd
from dodfminer.extract.polished.acts.aposentadoria import Retirements
import numpy as np
from itertools import zip_longest
from itertools import chain
from collections import Counter
import gc

In [79]:
# ls data/dodfs_flat/

In [80]:
def get_reverse_date(s):
    date = re.search( r'((\d+)[-](\d+)[-](\d+))', s)
    date = date.group()
    date = '-'.join(date.split('-')[::-1])
    return date

def get_num(s):
    return int(re.search('(\d+)', s).group())

In [81]:
df = pd.DataFrame()

In [172]:
sorted(dodfs_txt)

['data/dodfs_flat/DODF 001 01-01-2019 EDICAO ESPECIAL.txt',
 'data/dodfs_flat/DODF 001 02-01-2019 INTEGRA.txt',
 'data/dodfs_flat/DODF 002 03-01-2019 INTEGRA.txt',
 'data/dodfs_flat/DODF 002 08-01-2019 EDICAO EXTRA.txt',
 'data/dodfs_flat/DODF 003 04-01-2019 INTEGRA.txt',
 'data/dodfs_flat/DODF 003 11-01-2019 EDICAO EXTRA.txt',
 'data/dodfs_flat/DODF 004 07-01-2019 INTEGRA.txt',
 'data/dodfs_flat/DODF 004 07-01-2019 SUPLEMENTO.txt',
 'data/dodfs_flat/DODF 004 14-01-2019 EDICAO EXTRA.txt',
 'data/dodfs_flat/DODF 005 08-01-2019 INTEGRA.txt',
 'data/dodfs_flat/DODF 005 15-01-2019 EDICAO EXTRA.txt',
 'data/dodfs_flat/DODF 006 09-01-2019 INTEGRA.txt',
 'data/dodfs_flat/DODF 006 17-01-2019 EDICAO EXTRA.txt',
 'data/dodfs_flat/DODF 007 10-01-2019 INTEGRA.txt',
 'data/dodfs_flat/DODF 007 10-01-2019 SUPLEMENTO.txt',
 'data/dodfs_flat/DODF 007 21-01-2019 EDICAO EXTRA.txt',
 'data/dodfs_flat/DODF 008 11-01-2019 INTEGRA.txt',
 'data/dodfs_flat/DODF 008 22-01-2019 EDICAO EXTRA.txt',
 'data/dodfs_fl

## Quantos DODFs baixados?

In [82]:
dodfs_txt = glob.glob('data/dodfs_flat/*.txt')
df['file'] = [i.split('/')[-1] for i in dodfs_txt]
df['date'] = [get_reverse_date(s) for s in df.file]
# df['date'] = pd.to_datetime(df.date)
df['num'] = [get_num(s) for s in df.file]
df.drop('file', axis=1, inplace=True)
len(dodfs_txt)

595

## Quantos tiveram aposentadoria detectada?

In [83]:
%%time
dodfs_ext = [Retirements(i, 'regex') for i in dodfs_txt]
df['has_act'] = [not i.data_frame.empty for i in dodfs_ext] 
dodfs_ext = [i for i in dodfs_ext if not i.data_frame.empty]
print(
    "TOTAL DE ARQUIVOS COM APOSENTADORIAS:",
    df[~df.has_act].shape[0]
)

TOTAL DE ARQUIVOS COM APOSENTADORIAS: 368
CPU times: user 11.9 s, sys: 166 ms, total: 12.1 s
Wall time: 12.1 s


In [84]:
douro = pd.read_csv('data/aposentadoria-ouro/aposentaria-ouro-®.csv',
            sep='®', engine='python', index_col=False,)
douro.shape

(5516, 19)

In [85]:
douro.groupby(['DATA_DODF', 'NUM_DODF']).count().shape

(227, 17)

In [86]:
gp_ouro = douro.groupby(
    ['DATA_DODF', 'NUM_DODF']).count().iloc[:, 0:0].index
gp_miner = df[df.has_act].groupby(
    ['date', 'num']).count().iloc[:, 0:0].index

In [87]:
len(gp_miner), len(gp_ouro)

(226, 227)

In [88]:
sminer = set(gp_miner)
souro = set(gp_ouro)

In [89]:
MATTERS = ['DATA_DODF', 'NUM_DODF']
matters = ['date', 'num']
df_dtnum = df[df.has_act][matters].groupby(matters).count()
df_dtnum.reset_index(inplace=True)

douro_dtnum = douro[MATTERS].groupby(MATTERS).count()
douro_dtnum.reset_index(inplace=True)
douro_dtnum.rename({'DATA_DODF':'date', 'NUM_DODF':'num'},
                   inplace=True, axis=1)

In [90]:
souro_date = set(douro_dtnum.date.astype(str))
sdf_date = set(df_dtnum.date.astype(str))

In [118]:
duplis = list(
    zip_longest(douro_dtnum.values, df_dtnum.values,
                fillvalue=('empty', 'empty')
               )
)
eq = [i for i in duplis if np.array_equal(*i)]
neq = [i for i in duplis if not np.array_equal(*i)]
len(eq), len(neq)

(28, 199)

In [163]:
ctr= Counter([tuple(i) for i in chain(*neq)])
diff = {k:v for (k,v) in ctr.items() if v == 1}
eqq = {k:v for (k, v) in ctr.items() if v > 1}
diff

{('2018-04-19', 75): 1,
 ('2018-06-20', 116): 1,
 ('2018-07-11', 130): 1,
 ('2018-08-02', 146): 1,
 ('2018-08-17', 157): 1,
 ('2019-02-15', 33): 1,
 ('2019-02-26', 16): 1,
 ('2019-03-07', 44): 1,
 ('2019-03-15', 19): 1,
 ('2019-03-27', 58): 1,
 ('2019-04-03', 63): 1,
 ('2019-04-17', 30): 1,
 ('2019-07-03', 123): 1,
 ('2019-07-05', 125): 1,
 ('2019-07-23', 47): 1,
 ('2019-07-25', 139): 1,
 ('2019-08-30', 165): 1,
 ('2019-09-02', 166): 1,
 ('2019-09-10', 172): 1,
 ('empty', 'empty'): 1}

In [164]:
len(eqq)

189

In [165]:
eqq

{('2018-04-20', 76): 2,
 ('2018-04-23', 77): 2,
 ('2018-04-25', 79): 2,
 ('2018-04-26', 80): 2,
 ('2018-04-30', 82): 2,
 ('2018-05-02', 83): 2,
 ('2018-05-03', 84): 2,
 ('2018-05-04', 85): 2,
 ('2018-05-07', 86): 2,
 ('2018-05-08', 87): 2,
 ('2018-05-09', 88): 2,
 ('2018-05-11', 90): 2,
 ('2018-05-14', 91): 2,
 ('2018-05-15', 92): 2,
 ('2018-05-16', 93): 2,
 ('2018-05-18', 95): 2,
 ('2018-05-21', 96): 2,
 ('2018-05-28', 101): 2,
 ('2018-05-29', 102): 2,
 ('2018-05-30', 103): 2,
 ('2018-06-04', 104): 2,
 ('2018-06-05', 105): 2,
 ('2018-06-06', 106): 2,
 ('2018-06-07', 107): 2,
 ('2018-06-08', 108): 2,
 ('2018-06-11', 109): 2,
 ('2018-06-12', 110): 2,
 ('2018-06-13', 111): 2,
 ('2018-06-14', 112): 2,
 ('2018-06-18', 114): 2,
 ('2018-07-12', 131): 2,
 ('2018-07-13', 132): 2,
 ('2018-07-16', 133): 2,
 ('2018-07-18', 135): 2,
 ('2018-07-20', 137): 2,
 ('2018-07-24', 139): 2,
 ('2018-07-26', 141): 2,
 ('2018-07-27', 142): 2,
 ('2018-07-31', 144): 2,
 ('2018-08-01', 145): 2,
 ('2018-08-20', 1

In [136]:
len(diff)

20

### Portanto há 19 casos de descompasse:

1. arquivo foi extraído mas não está na base ouro
2. arquivo foi extraído mas não teve atos encontrados e está na base ouro
3. arquivo não foi extraído e não está na base ouro
4. arquivo não foi extraído e está na base ouro

Apenas os casos (2) e (4) interessam.

In [143]:
apenas_douro = {}
apenas_miner = {}
index_douro = douro_dtnum.set_index(['date', 'num']).index
index_miner = df_dtnum.set_index(['date', 'num']).index

for index in diff:
    if index in index_douro:
        apenas_douro[index] = index
    elif index in index_miner:
        apenas_miner[index] = index
    else:
        print("ugly:", index)
del index_douro, index_miner
gc.collect()

ugly: ('empty', 'empty')


7831

In [146]:
list(apenas_douro.values())

[('2018-04-19', 75),
 ('2018-08-02', 146),
 ('2018-08-17', 157),
 ('2019-02-26', 16),
 ('2019-03-07', 44),
 ('2019-03-15', 19),
 ('2019-04-03', 63),
 ('2019-07-03', 123),
 ('2019-07-05', 125),
 ('2019-09-02', 166)]

In [188]:
pd.concat(
    [df[(df.date == i[0]) & (df.num == i[1])]
     for i in apenas_douro.values()]
)

,date,num,has_act
218,2018-04-19,75,False
259,2018-08-02,146,False
368,2018-08-17,157,False
322,2019-02-26,16,False
95,2019-03-07,44,False
468,2019-03-07,44,False
120,2019-03-15,19,False
392,2019-04-03,63,False
7,2019-07-03,123,False
223,2019-07-03,123,False


**Vemos portanto que de fato há textos com ocorrências de concessão de aposentadoria e que ainda assim não foram detectados pelo DODFMiner.**

In [147]:
list(apenas_miner.values())

[('2018-06-20', 116),
 ('2018-07-11', 130),
 ('2019-02-15', 33),
 ('2019-03-27', 58),
 ('2019-04-17', 30),
 ('2019-07-23', 47),
 ('2019-07-25', 139),
 ('2019-08-30', 165),
 ('2019-09-10', 172)]

## Fazer dump das (datas , números) referente aos arquivos que devem ser lidos para plotagem dos gráficos.

In [ ]:
# douro[(douro.DATA_DODF == '2019-03-15') & (douro.NUM_DODF == 19)]

In [200]:
_ = pd.concat(
    [df[(df.date == i[0]) & (df.num == i[1])]
     for i in apenas_miner.values()]
)
_ = _[_.has_act]
_

,date,num,has_act
108,2018-06-20,116,True
487,2018-07-11,130,True
325,2019-02-15,33,True
116,2019-03-27,58,True
470,2019-04-17,30,True
546,2019-07-23,47,True
103,2019-07-25,139,True
271,2019-08-30,165,True
467,2019-09-10,172,True


In [209]:
# OBS: não está rigorosamente  correto no caso de
# ocorrer repetição de dada. Porém como isso não ocorreu,
# não houve problemas

l = []
for i in dodfs_ext:
    num = re.search('\d+', i._file_name)
    date = '-'.join(re.search(
        r'(\d+[-]\d+[-]\d+)', i._file_name).group().split('-')[::-1]
    )
    if date in _.date.values:
        print(date)
        l.append(i)

2019-07-25
2018-06-20
2019-03-27
2019-08-30
2019-02-15
2019-09-10
2019-04-17
2018-07-11
2019-07-23


In [214]:
l[2].data_frame

,Tipo do Ato,SEI,Nome,Matrícula,Tipo de Aposentadoria,Cargo,Classe,Padrao,Quadro,Fundamento Legal,Orgao,Vigencia,Matricula SIAPE
0,Aposentadoria,054.000.223/2010,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [215]:
l[2]._file_name

'data/dodfs_flat/DODF 058 27-03-2019 INTEGRA.txt'

In [217]:
df_bad = []
df_bad = pd.concat([i.data_frame for i in l])
df_bad

,Tipo do Ato,SEI,Nome,Matrícula,Tipo de Aposentadoria,Cargo,Classe,Padrao,Quadro,Fundamento Legal,Orgao,Vigencia,Matricula SIAPE
0,Aposentadoria,00080-\n00091971/2019-81,nan,nan,nan,Professor de Educacao Basica podera atuar em ...,vaga,nan,nan,nan,nan,nan,nan
0,Aposentadoria,431.001249/2016,da servidora ROSEMARI DE MELO SOUZA,104.727-2,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,Aposentadoria,054.000.223/2010,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,Aposentadoria,6.404/1976,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Aposentadoria,6.404/1976,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,Aposentadoria,10.887/2004,e artigos 1o e 15 da Lei n.o 10.887/2004; IV -...,n.o,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,Aposentadoria,467.000.117/2016,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,Aposentadoria,\n0040.002.239/2017,nan,nan,nan,nan,nan,nan,quadro seguinte: Plano Previdenciario\n(Passiv...,nan,nan,nan,nan
0,Aposentadoria,084.000522/2014,nan,nan,nan,Subsecretario(a) de Administracao Geral a com...,nan,nan,Quadro de Pessoal da Secretaria de\nEstado de ...,nan,Secretaria de\nEstado de Educacao,nan,nan
0,Aposentadoria,02.596.914/0001-92,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


**Ou seja: tudo errado**